In [1]:
import pandas as pd
import re
import warnings
from fuzzywuzzy import fuzz
warnings.filterwarnings('ignore')
pd.set_option('max_columns',151)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
outfield = pd.read_csv('data/2019-20_out.csv')
fpl = pd.read_csv('data/cleaned_players.csv')

In [3]:
for c in outfield.columns:
    print(f"'{c}'",end=',')

'player','nationality','position','squad','age','birth_year','games','games_starts','minutes','goals','assists','pens_made','pens_att','cards_yellow','cards_red','goals_per90','assists_per90','goals_assists_per90','goals_pens_per90','goals_assists_pens_per90','xg','npxg','xa','xg_per90','xa_per90','xg_xa_per90','npxg_per90','npxg_xa_per90','minutes_90s','shots_total','shots_on_target','shots_free_kicks','shots_on_target_pct','shots_total_per90','shots_on_target_per90','goals_per_shot','goals_per_shot_on_target','npxg_per_shot','xg_net','npxg_net','passes_completed','passes','passes_pct','passes_total_distance','passes_progressive_distance','passes_completed_short','passes_short','passes_pct_short','passes_completed_medium','passes_medium','passes_pct_medium','passes_completed_long','passes_long','passes_pct_long','xa_net','assisted_shots','passes_into_final_third','passes_into_penalty_area','crosses_into_penalty_area','progressive_passes','passes_live','passes_dead','passes_free_kicks'

In [4]:
attack = outfield.loc[:,['player','squad','position','games','minutes','minutes_90s',
                         'pens_made','pens_att','cards_yellow','cards_red',
                         'goals_per90','assists_per90','goals_assists_per90',
                         'goals_assists_pens_per90','xg_per90','xa_per90','xg_xa_per90',
                         'npxg_per90','npxg_xa_per90','xg_net']]
pos = ['FW','MF','MF,FW','FW,MF']

attack = attack[attack['position'].isin(pos)]
attack = attack[attack['minutes_90s']>=8].reset_index(drop=True)
attack

,player,squad,position,games,minutes,minutes_90s,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_assists_pens_per90,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,xg_net
0,Tammy Abraham,Chelsea,FW,34.0,2215.0,24.6,0.0,0.0,2.0,0.0,0.61,0.12,0.73,0.73,0.59,0.10,0.70,0.59,0.70,0.4
1,Che Adams,Southampton,FW,30.0,1111.0,12.3,0.0,0.0,0.0,0.0,0.32,0.16,0.49,0.49,0.34,0.10,0.44,0.34,0.44,-0.2
2,Sergio Agüero,Manchester City,FW,24.0,1452.0,16.1,2.0,3.0,1.0,0.0,0.99,0.19,1.18,1.05,0.85,0.15,1.00,0.71,0.86,2.2
3,Dele Alli,Tottenham,"MF,FW",25.0,1847.0,20.5,1.0,1.0,2.0,0.0,0.39,0.19,0.58,0.54,0.34,0.10,0.43,0.30,0.39,1.1
4,Miguel Almirón,Newcastle Utd,"MF,FW",36.0,3021.0,33.6,0.0,0.0,5.0,0.0,0.12,0.06,0.18,0.18,0.19,0.04,0.23,0.19,0.23,-2.4
5,Felipe Anderson,West Ham,"MF,FW",25.0,1497.0,16.6,0.0,0.0,1.0,0.0,0.06,0.24,0.30,0.30,0.15,0.18,0.32,0.15,0.32,-1.5
6,Michail Antonio,West Ham,FW,24.0,1768.0,19.6,1.0,1.0,3.0,0.0,0.51,0.15,0.66,0.61,0.59,0.14,0.73,0.55,0.69,-1.6
7,Stuart Armstrong,Southampton,MF,30.0,1715.0,19.1,0.0,0.0,4.0,0.0,0.26,0.16,0.42,0.42,0.15,0.06,0.21,0.15,0.21,2.2
8,Christian Atsu,Newcastle Utd,"FW,MF",19.0,749.0,8.3,0.0,0.0,2.0,0.0,0.00,0.36,0.36,0.36,0.02,0.11,0.13,0.02,0.13,-0.2
9,Pierre-Emerick Aubameyang,Arsenal,FW,36.0,3136.0,34.8,2.0,2.0,3.0,1.0,0.63,0.09,0.72,0.66,0.45,0.09,0.54,0.41,0.49,6.3


In [5]:
attack.shape

(184, 20)

In [6]:
fpl_pos = ['MID','FWD']
fpl_att = fpl[fpl['element_type'].isin(fpl_pos)]
fpl_att = fpl_att.loc[:,['first_name','second_name','total_points','now_cost']]
fpl_att['player'] = None
fpl_att['player'] = fpl_att['first_name'].str.cat(fpl_att['second_name'],sep=' ')
fpl_att = fpl_att.drop(columns=['first_name','second_name']).reset_index(drop=True)
fpl_att

,total_points,now_cost,player
0,0,69,Mesut Özil
1,12,120,Pierre-Emerick Aubameyang
2,14,85,Alexandre Lacazette
3,5,55,Granit Xhaka
4,2,78,Nicolas Pépé
5,0,49,Lucas Torreira
6,0,49,Joseph Willock
7,0,54,Reiss Nelson
8,9,59,Edward Nketiah
9,0,45,Emile Smith Rowe


In [7]:
cost = {}
points = {}

for att_p in attack.itertuples():
    flag=0
    for fpl_p in fpl_att.itertuples():
        if (fuzz.token_set_ratio(att_p[1],fpl_p[3])>75):
            points[att_p[1]] = fpl_p[1]
            cost[att_p[1]] = fpl_p[2]/10
            flag=1
            break

In [8]:
attack['cost'] = attack['player'].map(cost)
attack['points'] = attack['player'].map(points)
attack = attack.reset_index(drop=True)
attack

,player,squad,position,games,minutes,minutes_90s,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_assists_pens_per90,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,xg_net,cost,points
0,Tammy Abraham,Chelsea,FW,34.0,2215.0,24.6,0.0,0.0,2.0,0.0,0.61,0.12,0.73,0.73,0.59,0.10,0.70,0.59,0.70,0.4,7.4,1.0
1,Che Adams,Southampton,FW,30.0,1111.0,12.3,0.0,0.0,0.0,0.0,0.32,0.16,0.49,0.49,0.34,0.10,0.44,0.34,0.44,-0.2,5.9,3.0
2,Sergio Agüero,Manchester City,FW,24.0,1452.0,16.1,2.0,3.0,1.0,0.0,0.99,0.19,1.18,1.05,0.85,0.15,1.00,0.71,0.86,2.2,10.4,0.0
3,Dele Alli,Tottenham,"MF,FW",25.0,1847.0,20.5,1.0,1.0,2.0,0.0,0.39,0.19,0.58,0.54,0.34,0.10,0.43,0.30,0.39,1.1,7.8,1.0
4,Miguel Almirón,Newcastle Utd,"MF,FW",36.0,3021.0,33.6,0.0,0.0,5.0,0.0,0.12,0.06,0.18,0.18,0.19,0.04,0.23,0.19,0.23,-2.4,6.0,5.0
5,Felipe Anderson,West Ham,"MF,FW",25.0,1497.0,16.6,0.0,0.0,1.0,0.0,0.06,0.24,0.30,0.30,0.15,0.18,0.32,0.15,0.32,-1.5,6.4,2.0
6,Michail Antonio,West Ham,FW,24.0,1768.0,19.6,1.0,1.0,3.0,0.0,0.51,0.15,0.66,0.61,0.59,0.14,0.73,0.55,0.69,-1.6,6.4,8.0
7,Stuart Armstrong,Southampton,MF,30.0,1715.0,19.1,0.0,0.0,4.0,0.0,0.26,0.16,0.42,0.42,0.15,0.06,0.21,0.15,0.21,2.2,5.5,2.0
8,Christian Atsu,Newcastle Utd,"FW,MF",19.0,749.0,8.3,0.0,0.0,2.0,0.0,0.00,0.36,0.36,0.36,0.02,0.11,0.13,0.02,0.13,-0.2,8.4,0.0
9,Pierre-Emerick Aubameyang,Arsenal,FW,36.0,3136.0,34.8,2.0,2.0,3.0,1.0,0.63,0.09,0.72,0.66,0.45,0.09,0.54,0.41,0.49,6.3,12.0,12.0


In [9]:
attack[attack.isna().any(axis=1)]

,player,squad,position,games,minutes,minutes_90s,pens_made,pens_att,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_assists_pens_per90,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,xg_net,cost,points
14,Nabil Bentaleb,Newcastle Utd,MF,12.0,751.0,8.3,0.0,0.0,2.0,0.0,0.00,0.00,0.00,0.00,0.01,0.03,0.04,0.01,0.04,-0.1,NaN,NaN
18,Philip Billing,Bournemouth,MF,34.0,2530.0,28.1,0.0,0.0,8.0,0.0,0.04,0.04,0.07,0.07,0.09,0.04,0.13,0.09,0.13,-1.5,NaN,NaN
23,Emi Buendía,Norwich City,"FW,MF",36.0,2464.0,27.4,0.0,0.0,6.0,1.0,0.04,0.26,0.29,0.29,0.08,0.26,0.35,0.08,0.35,-1.3,NaN,NaN
25,Todd Cantwell,Norwich City,"FW,MF",37.0,2489.0,27.7,0.0,0.0,6.0,0.0,0.22,0.07,0.29,0.29,0.21,0.10,0.31,0.21,0.31,0.1,NaN,NaN
26,Étienne Capoue,Watford,MF,30.0,2627.0,29.2,0.0,0.0,8.0,0.0,0.00,0.10,0.10,0.10,0.03,0.10,0.13,0.03,0.13,-0.8,NaN,NaN
28,Nathaniel Chalobah,Watford,MF,22.0,930.0,10.3,0.0,0.0,1.0,0.0,0.00,0.10,0.10,0.10,0.05,0.12,0.17,0.05,0.17,-0.5,NaN,NaN
30,Tom Cleverley,Watford,MF,18.0,971.0,10.8,0.0,0.0,2.0,0.0,0.09,0.00,0.09,0.09,0.14,0.05,0.19,0.14,0.19,-0.6,NaN,NaN
32,Lewis Cook,Bournemouth,MF,27.0,1352.0,15.0,0.0,0.0,5.0,0.0,0.00,0.00,0.00,0.00,0.02,0.06,0.08,0.02,0.08,-0.3,NaN,NaN
36,Troy Deeney,Watford,FW,27.0,2225.0,24.7,5.0,6.0,3.0,0.0,0.40,0.08,0.49,0.28,0.40,0.14,0.53,0.21,0.35,0.2,NaN,NaN
38,Gerard Deulofeu,Watford,FW,28.0,2098.0,23.3,1.0,1.0,3.0,0.0,0.17,0.21,0.39,0.34,0.22,0.20,0.42,0.19,0.39,-1.2,NaN,NaN


In [10]:
attack = attack.dropna().reset_index(drop=True)
attack['ppg'] = attack.apply(lambda row: (row['points']/row['games']),axis=1)
attack['ppc'] = attack.apply(lambda row: (row['points']/row['cost']),axis=1)

In [11]:
attack.shape

(153, 24)

In [12]:
attack['squad'].value_counts()

West Ham           11
Manchester Utd     11
Everton            10
Tottenham          10
Brighton            9
Leicester City      9
Arsenal             9
Burnley             9
Chelsea             9
Sheffield Utd       8
Liverpool           8
Manchester City     8
Southampton         8
Aston Villa         8
Newcastle Utd       8
Crystal Palace      7
Wolves              6
Bournemouth         3
Watford             1
Norwich City        1
Name: squad, dtype: int64

In [13]:
attack.to_csv('data/attacker_dash.csv',index=False)